# Travel Demand Model Data Inputs
> 

## Setup 

In [ ]:
import pandas as pd
import pathlib
import os
from utils import *

In [ ]:
# current working directory
local_path = pathlib.Path().absolute()
# set workspace
arcpy.env.workspace = os.path.join(local_path, 'Workspace.gdb')
# overwrite true
arcpy.env.overwriteOutput = True

#### Notes
* old inputs: F:\Research and Analysis\misc\Reid_Haefer\Model\model_update_2018\data_inputs\lodging_occupancy
* Final inputs to produce: F:\Transportation\model\scenario_base\zonal

### Base Data

In [ ]:
# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)

# place holder URL - Demographics URL will have a new Feature Service titled "Block Group 2022 - Travel Demand Model Inputs"
census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/1'
sdf_census = get_fs_data_spatial(census_url)

# Get VHR data - this is current, no way to get historical data outside of the City of South Lake Tahoe
vhr_url = 'https://maps.trpa.org/server/rest/services/VHR/MapServer/0'
sdf_vhr = get_fs_data_spatial(vhr_url)

# Get Cumulative Accounting Unit data
units_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_units = get_fs_data_spatial_query(units_url, "Year = 2022")


In [ ]:
# Get Cumulative Accounting Unit data
units_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_units = get_fs_data_spatial_query(units_url, "Year = 2022")

sdf_units.to_csv('cumulative_accounting_units_2022.csv', index=False)

In [ ]:
sdf_units.spatial.to_featureclass(os.path.join(local_path, 'Workspace.gdb','cumulative_accounting_units_2022'))

## Socio Econ
> TAZ by total_residential_units,census_occ_rate,total_occ_units,occ_units_low_inc,occ_units_med_inc,occ_units_high_inc,persons_per_occ_unit,total_persons,emp_retail,emp_srvc,emp_rec,emp_game,emp_other

In [ ]:
units_url = ''
sdf_res = get_fs_data_spatial_query(units_url, "YEAR = 2022")

### Employment

> Purpose: Assign employees to TAZs
* Data Axle data and CBP data sources

In [ ]:
employment_url = ''


### Income by Residential Unit

> Purpose: Calculate #/% of residential units in low/mid/high income in each TAZ
* Get # of HH in each income bin by census block group from ACS
* Combine to low/mid/high income, with breaks at 60k and 100k
* Assign to TAZ


In [ ]:
# place holder URL - Demographics URL will have a new Feature Service titled "Block Group 2022 - Travel Demand Model Inputs"
census_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/1'
sdf_census = get_fs_data_spatial(census_url)


## Overnight Visitation
>TAZ by hotelmotel,resort,casino,campground,percentHouseSeasonal,beach

### Tourist Accommodation Units

In [ ]:
# Get VHR data - this is current, no way to get historical data outside of the City of South Lake Tahoe
vhr_url = 'https://maps.trpa.org/server/rest/services/VHR/MapServer/0'
sdf_vhr = get_fs_data_spatial(vhr_url)

# Get Tourist Accomodation Units data
tau_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_tau = get_fs_data_spatial_query(tau_url, "Year = 2022")


In [ ]:
# spatial join TAZ to VHR


### Campgrounds

> Purpose: calculate campground occupancy on model day and assign to TAZs
* in-process data: "F:\Research and Analysis\Transportation\Travel_Demand_Model\2023 Update\Input Data\Campgrounds\Campground_Visitation.xlsx"
* USFS data: https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_RecreationAreaActivities_01/MapServer/0

In [ ]:
# Get the data
campgroundu_url = 'https://maps.trpa.org/server/rest/services/Recreation/MapServer/1'
# campgrounds
sdf_campgrounds = get_fs_data_spatial_query(campgroundu_url, "RECREATION_TYPE = 'Campground")

# campground visitation
df_campground_visitation = pd.read_csv('2022/Data/Campground_Visitation.csv')

In [ ]:
# CA SP info https://csparks.maps.arcgis.com/apps/webappviewer/index.html?id=f96a883ff4154455b23bdc119f4574a9
# CA SP visitation

In [3]:
import pandas as pd
import requests
# get data params with Mason's API key
campsite_url = 'https://ridb.recreation.gov/api/v1/facilities/232768/campsites'
api = '45310cc1-df4f-4824-88ba-8bd75d06f57e'
params = {'apikey': api}

# rec data from https://ridb.recreation.gov/download
# https://ridb.recreation.gov/downloads/reservations2022.zip
# local copy
rec_gov = r"C:\Users\mbindl\Downloads\reservations2022\FY22 Historical Reservations Full.csv"
df_res = pd.read_csv(rec_gov)

C:\Users\mbindl\AppData\Local\Temp\ipykernel_25080\2368236116.py:12: DtypeWarning: Columns (5,6,7,11,12,13,15,17,27,29,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res = pd.read_csv(rec_gov)


In [10]:
# empty data frame to store campsite info
df_campsite_info_all = pd.DataFrame(columns=['Campground','Campsites', 'Total Nights', 'Total Visitors'])

# get campsite info from recreation.gov api
campgound_ids = {'232768':'Nevada Beach Campground',
                 '232769':'Fallen Leaf Campground',
                 '232876':'Meeks Bay Campground',
                 '232874':'William Kent Campground',
                 '232875':'Kaspian Campground',}

# create loop of campgrounds ids and get requests and count campsites
for campsite_id, campsite_name in campgound_ids.items():
    campsite_url = f'https://ridb.recreation.gov/api/v1/facilities/{campsite_id}/campsites'
    # get data for "Nevada Beach Campground"
    campsite_data = requests.get(campsite_url, params=params).json()
    # parse the data into a dataframe
    df_campsite = pd.DataFrame(campsite_data['RECDATA'])
    # count campsites
    print(f'{campsite_name} has {df_campsite.CampsiteName.count()} campsites')
    site_count = df_campsite.CampsiteName.count()

    # filter by campground id from the downloaded data
    df = df_res[df_res.facilityid == int(campsite_id)]
    # filter to usetype = Overnight
    df = df[df.usetype == 'Overnight']
    # filter by August 2022
    df_august22 = df[df.startdate.str.contains('2022-08')]
    # # filter out cancelled reservations
    # df_august22 = df_august22[df_august22.status != 'Cancelled']
    # # filter out reservations with 0 nights
    # df_august22 = df_august22[df_august22.nights != '0']
    # filter out reservations with end date in September 2022
    df_august22 = df_august22[~df_august22.enddate.str.contains('2022-09')]
    # use .loc to strip off ' days' from the nights column
    df_august22.loc[:, 'nights'] = df_august22.nights.map(lambda x: x.rstrip(' days'))
    # use .loc to strip off ' day' from the nights column
    df_august22.loc[:, 'nights'] = df_august22.nights.map(lambda x: x.rstrip(' day'))
    # drop a specific row based on a value in nights column
    df_august22 = df_august22[df_august22.nights != '00:00:00']
    # convert nights column to integer
    df_august22.nights = df_august22.nights.astype(int)

    # number of nights reserved for Nevada Beach Campground in August 2022
    total_nights = df_august22.nights.sum()
    # get total number of people reserved for Nevada Beach Campground in August 2022
    total_people = df_august22.numberofpeople.sum()
    print(f'{campsite_name} had {total_nights} nights reserved and {total_people} total visitors during August 2022')
    # append as row to new dataframe with campsite name, total nights, and total people
    df_campsite_info = pd.DataFrame([[campsite_name, site_count, total_nights, total_people]], 
                                    columns=['Campground', 'Campsites','Total Nights', 'Total Visitors'])
    
    # Check if 'df_campsite_info_all' exists in the local namespace
    if 'df_campsite_info_all' in locals():
        # If it exists, append 'df_campsite_info' to it as a new row
        df_campsite_info_all = pd.concat([df_campsite_info_all, df_campsite_info], ignore_index=True)
    else:
        # If it doesn't exist, assign 'df_campsite_info' to it directly
        df_campsite_info_all = df_campsite_info.copy()

Nevada Beach Campground has 52 campsites
Nevada Beach Campground had 1586 nights reserved and 2082.0 total visitors during August 2022
Fallen Leaf Campground has 196 campsites
Fallen Leaf Campground had 7501 nights reserved and 11255.0 total visitors during August 2022
Meeks Bay Campground has 40 campsites
Meeks Bay Campground had 1572 nights reserved and 2678.0 total visitors during August 2022
William Kent Campground has 79 campsites
William Kent Campground had 2807 nights reserved and 4336.0 total visitors during August 2022
Kaspian Campground has 9 campsites
Kaspian Campground had 327 nights reserved and 561.0 total visitors during August 2022


In [20]:
# create loop of campgrounds ids and get requests and count campsites
for campsite_id, campsite_name in campgound_ids.items():
    campsite_url = f'https://ridb.recreation.gov/api/v1/facilities/{campsite_id}/campsites'
    # get data for "Nevada Beach Campground"
    campsite_data = requests.get(campsite_url, params=params).json()
    # parse the data into a dataframe
    df_campsite = pd.DataFrame(campsite_data['RECDATA'])
    # count campsites
    print(f'{campsite_name} has {df_campsite.CampsiteName.count()} campsites')
    site_count = df_campsite.CampsiteName.count()

    # filter by campground id from the downloaded data
    df = df_res[df_res.facilityid == int(campsite_id)]
    # filter to usetype = Overnight
    df = df[df.usetype == 'Overnight']
    # filter by August 2022
    df_august22 = df[df.startdate.str.contains('2022-08')]

Nevada Beach Campground has 52 campsites
Fallen Leaf Campground has 196 campsites
Meeks Bay Campground has 40 campsites
William Kent Campground has 79 campsites
Kaspian Campground has 9 campsites


In [23]:
df_august22.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 8210810 to 8211478
Data columns (total 35 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   historicalreservationid  153 non-null    object 
 1   ordernumber              153 non-null    object 
 2   agency                   153 non-null    object 
 3   orgid                    153 non-null    int64  
 4   codehierarchy            153 non-null    object 
 5   regioncode               153 non-null    object 
 6   regiondescription        153 non-null    object 
 7   parentlocationid         153 non-null    object 
 8   parentlocation           153 non-null    object 
 9   legacyfacilityid         153 non-null    float64
 10  park                     153 non-null    object 
 11  sitetype                 153 non-null    object 
 12  usetype                  153 non-null    object 
 13  productid                153 non-null    object 
 14  inventorytype        

In [13]:
df_campsite_info_all


,Campground,Campsites,Total Nights,Total Visitors
0,Nevada Beach Campground,52,1586,2082.0
1,Fallen Leaf Campground,196,7501,11255.0
2,Meeks Bay Campground,40,1572,2678.0
3,William Kent Campground,79,2807,4336.0
4,Kaspian Campground,9,327,561.0


In [19]:
df

,historicalreservationid,ordernumber,agency,orgid,codehierarchy,regioncode,regiondescription,parentlocationid,parentlocation,legacyfacilityid,...,totalbeforetax,discount,totalpaid,startdate,enddate,orderdate,nights,numberofpeople,equipmentdescription,equipmentlength
8210774,65499887-16f4-4381-b8ee-4dcc89f7c47b,0300355902-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,0.0,0.0,0.0,2021-10-08T00:00:00Z,2021-10-10T00:00:00Z,2021-08-21T19:34:16.047908Z,2 days,6.0,Tent,0.0
8210775,00a0aded-a902-4f17-bbc4-533edf7f566a,0306671463-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,18.0,0.0,18.0,2021-10-07T00:00:00Z,2021-10-08T00:00:00Z,2021-06-20T18:53:59.47631Z,1 day,2.0,Tent,0.0
8210776,9107d259-cd77-40c4-b46f-9b9822c8aa18,0307728257-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,0.0,0.0,0.0,2021-10-01T00:00:00Z,2021-10-03T00:00:00Z,2021-07-09T16:08:45.497455Z,2 days,6.0,Tent,0.0
8210777,40a79493-3610-4754-ae2a-6926b2690151,0317278204-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,0.0,0.0,0.0,2021-10-02T00:00:00Z,2021-10-03T00:00:00Z,2021-09-09T00:51:38.893845Z,1 day,1.0,Tent,0.0
8210778,2cbb2754-129a-4a34-a1e6-30dd119a0057,0318380050-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,0.0,0.0,0.0,2021-10-08T00:00:00Z,2021-10-11T00:00:00Z,2021-07-15T18:24:33.64354Z,3 days,5.0,Tent,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8211474,65610e3c-61d4-40eb-b8db-758f2fab4be1,0499474798-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,77.0,0.0,77.0,2022-06-16T00:00:00Z,2022-06-19T00:00:00Z,2022-01-01T15:07:59.295729Z,3 days,6.0,Tent,0.0
8211475,fb5370ad-0fd9-4836-8f5d-c81bff2310d0,0499670335-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,31.0,0.0,31.0,2022-06-23T00:00:00Z,2022-06-24T00:00:00Z,2022-05-21T03:57:06.747053Z,1 day,2.0,Tent,0.0
8211476,696789db-32bd-4627-9be6-12b227b228b2,0499730859-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,54.0,-23.0,31.0,2022-08-27T00:00:00Z,2022-08-29T00:00:00Z,2022-04-18T02:46:04.22146Z,2 days,1.0,Tent,0.0
8211477,77092148-178c-4f8b-b46c-b06e869940cd,0499850477-1,USFS,131,|1|70903|72602|74417|71663,R05,Pacific Southwest Region,74839,Lake Tahoe Basin Management Unit,71663.0,...,18.0,0.0,18.0,2022-05-31T00:00:00Z,2022-06-03T00:00:00Z,2022-05-16T17:24:08.473836Z,3 days,6.0,Tent,0.0


In [ ]:
# remove any reservation that has an enddate in september
df_res_august = df_res[~df_res.enddate.str.contains('2022-09')]

## Occupancy Rates
> TAZ by hotelmotel,resort,casino,campground,house,seasonal

### Lodging Occupancy


> Purpose: calculate occupancy rates for hotels and VHRs and assign to TAZs

* Placer has Occupancy data at the TOT tax district. Josh will get the spatial file from Placer County GIS. 
* City of South Lake has hotel level occupancy data that needs to be converted to a spatial file. Reports live here: https://www.cityofslt.us/805/Zone-Detail-Reports 
* Douglas County has occupancy rates from the Casino reports: https://gaming.nv.gov/about/abstract/report/
* Washoe County has occupancy rates at the District level from RSCVA (single number by type of occupancy) "F:\Research and Analysis\Visitation\occupancy\Washoe B Occupied Rooms by Market Segment - 2022.xlsx"
* Rest of El Dorado County? is only VHRs, estimating Occupancy Rates based on TOT rates F:\Research and Analysis\Visitation\occupancy

In [ ]:
# Get VHR data - this is current, no way to get historical data outside of the City of South Lake Tahoe
vhr_url = 'https://maps.trpa.org/server/rest/services/VHR/MapServer/0'
sdf_vhr = get_fs_data_spatial(vhr_url)

# Get Tourist Accomodation Units data
tau_url = 'https://maps.trpa.org/server/rest/services/Existing_Development/MapServer/2'
sdf_tau = get_fs_data_spatial_query(tau_url, "Year = 2022")

# get occupancy rate data - see above for sources


### Housing Occupancy


> Purpose: estimate household size in each occupied housing unit
* get total Residential Units aggregated to TAZ
* get total VHRs aggregated to TAZ
* apply occupancy rate from ACS
* ACS 2022 by Block Group - household size

In [ ]:
# Get Houshold Block Group data
household_url = ''
sdf_household = get_fs_data_spatial(household_url)

## School Enrollment
> TAZ by elementary_school_enrollment,middle_school_enrollment,high_school_enrollment,college_enrollment

> Purpose: Collate school enrollment files
* create spatial file 

In [ ]:
# Get School Enrollment data
school_url_table     = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/32'
df_school_enrollment = get_fs_data(school_url_table)

# Get School Enrollment data - spatial
school_url_spatial = 'https://maps.trpa.org/server/rest/services/Datadownloader_PlanningandJurisdictions/MapServer/14'
sdf_school         = get_fs_data_spatial(school_url_spatial)

# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)

In [ ]:
# set Type to Null
sdf_school['TYPE'] = None
# set SchoolType to 'elementary' if it contains 'elementary' or 'magnet' or 'academy'
sdf_school.loc[sdf_school['NAME'].str.contains('elementary', case=False), 'TYPE'] = 'Elementary School'
# set SchoolType to 'middle' if it contains 'middle'
sdf_school.loc[sdf_school['NAME'].str.contains('middle', case=False), 'TYPE'] = 'Middle School'
# set SchoolType to 'high' if it contains 'high'
sdf_school.loc[sdf_school['NAME'].str.contains('high', case=False), 'TYPE'] = 'High School'
# set SchoolType to 'college' if it contains 'college'
sdf_school.loc[sdf_school['NAME'].str.contains('college', case=False), 'TYPE'] = 'College'
# set SchoolType to 'other' if it it does not contain any of the above
sdf_school.loc[sdf_school['TYPE'].isnull(), 'TYPE'] = 'Elementary School'

In [ ]:
# spatial join TAZs to School points
sdf_school_taz = sdf_school.spatial.join(sdf_taz, how='inner')
# group by TYPE and sum of Enrollment within TAZ 
sdf_school_taz_grouped = sdf_school_taz.groupby(['TYPE', 'TAZ']).agg(
                                                {'ENROLLMENT': 'sum'}).reset_index()
# unstack by TYPE as columns and TAZ as a column
sdf_school_taz_grouped_pivot = sdf_school_taz_grouped.pivot(index='TAZ', 
                                                            columns='TYPE', 
                                                            values='ENROLLMENT').reset_index()
# merge to sdf_taz to get all tazs
sdf_taz_school = pd.merge(sdf_taz, sdf_school_taz_grouped_pivot, how='left', on='TAZ')
# drop SHAPE column
sdf_taz_school = sdf_taz_school.drop(columns='SHAPE')
# fill NA with 0 for all rows
sdf_taz_school = sdf_taz_school.fillna(0)
# cast all fields to int
sdf_taz_school = sdf_taz_school.astype(int)
# rename columns
sdf_taz_school.rename(columns={'Elementary School':'elementary_school_enrollment',
                               'Middle School':'middle_school_enrollment',
                               'High School':'high_school_enrollment',
                               'College':'college_enrollment'}, inplace=True)

# export to csv
sdf_taz_school.to_csv(os.path.join('SchoolEnrollment.csv'), index=False)

In [ ]:
sdf_taz_school

#### other way 

In [ ]:
df_school_enrollment_22 = df_school_enrollment[df_school_enrollment['Year'] == '2022-2023']
# Add a row for LTCC - Lake Tahoe Community College
ltcc = {'School_Name': 'Lake Tahoe Community College', 'Level_': 'College', 'Enrollment': 2909}
df_school_enrollment_22 = pd.concat([df_school_enrollment_22, pd.DataFrame([ltcc])], ignore_index=True)
# join school spatial to school table
sdf_school_enroll = pd.merge(sdf_school, df_school_enrollment, left_on='SchoolID', how='left')

In [ ]:
# Get TAZ data
taz_url = 'https://maps.trpa.org/server/rest/services/Transportation_Planning/MapServer/6'
sdf_taz = get_fs_data_spatial(taz_url)

# Get School Enrollment data - spatial
school_url_spatial = 'https://maps.trpa.org/server/rest/services/Datadownloader_PlanningandJurisdictions/MapServer/14'
sdf_school         = get_fs_data_spatial(school_url_spatial)


# keep only the columns we need; SHAPE and TAZ for TAZ and SHAPE and ENROLLMENT for schools
sdf_taz = sdf_taz[['TAZ', 'SHAPE']]
sdf_school = sdf_school[['ENROLLMENT', 'SHAPE']]

# convert to feature class
sdf_taz.spatial.to_featureclass(location=os.path.join(arcpy.env.workspace, 'TAZ'), overwrite=True, sanitize_columns=False)
sdf_school.spatial.to_featureclass(location=os.path.join(arcpy.env.workspace, 'Schools'), overwrite=True, sanitize_columns=False)

In [ ]:
field_mappings = arcpy.FieldMappings()
# add tables to field mappings
field_mappings.addTable('TAZ')
field_mappings.addTable('Schools')


# print names of fields in field mappings
for field in field_mappings.fields:
    print(field.name)

In [ ]:
# create field mappings
field_mappings = arcpy.FieldMappings()

# add tables to field mappings
field_mappings.addTable('TAZ')
field_mappings.addTable('Schools')

# for the output.
enrollment = field_mappings.findFieldMapIndex("ENROLLMENT")
fieldmap = field_mappings.getFieldMap(enrollment)
 
# Get the output field's properties as a field object
field = fieldmap.outputField
 
# Rename the field and pass the updated field object back into the field map
field.name = "Total_Enrollment"
field.aliasName = "Total Enrollment"
fieldmap.outputField = field
 
# Set the merge rule to mean and then replace the old fieldmap in the mappings object
# with the updated one
fieldmap.mergeRule = "sum"
field_mappings.replaceFieldMap(enrollment, fieldmap)

# spatial join that sums enrollment for each TAZ
arcpy.analysis.SpatialJoin(
    target_features   =sdf_taz,
    join_features     =sdf_school,
    out_feature_class ="TAZ_School_Enrollment",
    join_operation    ="JOIN_ONE_TO_ONE",
    join_type         ="KEEP_ALL",
    field_mapping     =field_mappings,
    match_option      ="INTERSECT",
)

In [ ]:
# get the data 
sdf_taz_school = pd.DataFrame.spatial.from_featureclass("TAZ_School_Enrollment")



## Forecasts
> Forecast 2040 and 2050 development and population change

In [ ]:
# ??